In [1]:
from optimize import *


Starting MATLAB engine...
-------------------------
Elapsed: 33.68 sec


In [ ]:
result = optimize_multiple(9, 400, seeds=[0], functions={'forest': forest_minimize}, verb=True)

Seed: 0
forest:
Iteration No: 1 started. Evaluating function at random point.
Iteration No: 1 ended. Evaluation done at random point.
Time taken: 373.9062
Function value obtained: 0.6451
Current minimum: 0.6451
Iteration No: 2 started. Evaluating function at random point.


## Evaluate Optimum

In [5]:
from skopt import load
def objective():
    pass
scenario = 2
best_opt = "forest"
max_iters = 50
optimum = res_optimum_mult([load("./optims/scenario"+str(scenario)+"/"+best_opt+"_"+str(seed)+"_"+str(max_iters)+".opt") for seed in range(5)])
print("optimum:")
print(optimum)

function = full_model.run_model
dataset = "Coil20.mat"
ep_prtr = 201
ep = 101
arg_names = ["lr_pretrain", "lr", "alpha1", "maxIter1", "lambda1", "lambda2", "alpha2", "maxIter2"]
arg_vals = optimum[0]

args = dict(zip(arg_names, arg_vals))
data_loaded = loadmat("./saved/processed/" + dataset)
images_norm = data_loaded['X']
labels = data_loaded['Y'].reshape(-1)

total = 1.0
for seed in range(100, 105):
    print(seed)
    value = 1-function(images_norm, labels, seed, ep_prtr, ep, **args, verbose=False)[0]
    print("value:", value)
    total *= value

print("AVERAGE:", total**0.2)

optimum:
([0.005723346377343881, 0.0006960540795460365, 1.0686885283416514, 85, 2.527546354637533, 100], 0.725669763942158)


NameError: name 'full_model' is not defined